# Tutorial

Este tutorial busca servir de una guia para la creación de clases utilizando *uttrs*.

<span style="color: red"> DRAFT!</span>

## Versión interactiva

Puede ejecutarse este mismo tutorial de manera interactiba en Binder.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/quatrope/uttrs/HEAD?filepath=%2Fdocs%2Fsource%2Ftutorial.ipynb)

## Imports

En primer lugar es necesario importar todas las librerias que vamos a utilizar. En general, son solo tres:

- `attr` (*attrs*) que es la libreria en la cual se basa uttrs para crear clases con menos boiler plate.
- `astropy.units` La cual contiene todo el marco de utilidades para tratar con unidades físicas/astronómicas.
- `uttr` (*uttrs*) La librería que corresponde este tutorial.


> Nota: Este tutorial asume un conocimiento sobre estas librerías, una serie de enlaces de referencias pueden encontrarse al final de la página.

In [1]:
import attr
import uttr

import astropy.units as u

### The galaxy class

La clase que vamos a crear, consiste en una simplicación de la clase Galaxia del proyecot [Galaxy-Chop](https://github.com/vcristiani/galaxy-chop). 

Solo tiene 8 atributos y solo los primeros 7 tienen unidades y por lo tanto seran implementadas con la funcion `uttr.ib` de la librería. Estos son`x`, `y`, `z` son las posiciones de las particulas/estrellas en KiloParsecs (`kpc`) ; `vx`, `vy`,  `vz` las velocidades correspondientes a las particulas ($Km/s$); `m` su masa en masas solares ($M_\odot$). Todos

Finalmente `notes` texto libre sobre las galaxias y pueden ser implementadas con la librería *attrs* estandar

In [2]:
@attr.s
class Galaxy:
    x = uttr.ib(unit=u.kpc)
    y = uttr.ib(unit=u.kpc)
    z = uttr.ib(unit=u.kpc)

    vx = uttr.ib(unit=u.km / u.s)
    vy = uttr.ib(unit=u.km / u.s)
    vz = uttr.ib(unit=u.km / u.s)

    m = uttr.ib(unit=u.M_sun)

    notes = attr.ib(validator=attr.validators.instance_of(str))

### Galaxy with default units

Finalmente con la clase ya disponible podemos proceder a crear un objeto del tipo *Galaxy*,

Por cuestiones de simplicidadad, vamos a asumir solo 4 particulas con numeros totalmente arbitrarios en cada atributo.

Parte de la utilidad de usar uttrs, es la capacidad que tiene la libreria para agregar unidades por defecto automáticamente, o validar que la unidad ingresada sea equivalente.

Empecemos con un objeto en el cual todas las unidades se asignan autoḿaticamente

In [3]:
gal = Galaxy(
    x=[1, 1, 3, 4],
    y=[10, 2, 3, 100],
    z=[1, 1, 1, 1],
    vx=[1000, 1023, 2346, 1334],
    vy=[9956, 833, 954, 1024],
    vz=[1253, 956, 1054, 3568],
    m=[200, 100, 20, 5],
    notes="a random galaxy made with random numbers",
)

Si nos fijamos en cualquier atributo  de la clase, vamos a ver que todas las unidades se agregaron de manera correcta

In [4]:
gal.x

<Quantity [1., 1., 3., 4.] kpc>

In [5]:
gal.y

<Quantity [ 10.,   2.,   3., 100.] kpc>

In [6]:
gal.vx

<Quantity [1000., 1023., 2346., 1334.] km / s>

In [7]:
gal.m

<Quantity [200., 100.,  20.,   5.] solMass>

In [8]:
gal.notes

'a random galaxy made with random numbers'

### Galaxy with explicit units

Otra alternativa consiste en proveer unidades compatibles con las establecidas, hay que tener en cuenta que estas unidades tienen que ser equivalentes a las propuestas en la creacion de la clase. 

Por ejemplo podemos sugerir que la dimension `Z` este dada en parsecs, `vy` en $Km/h$ y las masas en $Kg$

In [9]:
gal = Galaxy(
    x=[1, 1, 3, 4],
    y=[10, 2, 3, 100],
    z=[1000, 1000, 1000, 1000] * u.parsec,
    vx=[1000, 1023, 2346, 1334],
    vy=[9956, 833, 954, 1024] * (u.km / u.h),
    vz=[1253, 956, 1054, 3568],
    m=[200, 100, 20, 5] * u.kg,
    notes="a random galaxy made with random numbers",
)

Como se nota en el ejemplo, esto funciona perfectamente, y ningun error aparece. Es mas podemos acceder a culquiera de los atributos presentes y todos mantienen las unidades sugeridas o explicitas

In [10]:
gal.z  # parsecs

<Quantity [1000., 1000., 1000., 1000.] pc>

In [11]:
gal.m  # kg

<Quantity [200., 100.,  20.,   5.] kg>

In [12]:
gal.vx  # default km/s

<Quantity [1000., 1023., 2346., 1334.] km / s>

In [13]:
gal.vy # km/h

<Quantity [9956.,  833.,  954., 1024.] km / h>

Por otro lado si por error cargamos un valor que tiene una unidad  que no es equivalente a la sugerida, se lanza un error de valor (`ValueError`)

Para demostrar esto vamos a extender el ejemplo, tratando de asignar a `x` un valor expresado en gramos (`g`)

In [14]:
gal = Galaxy(
    x=[1, 1, 3, 4] * u.g,
    y=[10, 2, 3, 100],
    z=[1000, 1000, 1000, 1000] * u.parsec,
    vx=[1000, 1023, 2346, 1334],
    vy=[9956, 833, 954, 1024] * (u.km / u.h),
    vz=[1253, 956, 1054, 3568],
    m=[200, 100, 20, 5] * u.kg,
    notes="a random galaxy made with random numbers",
)

ValueError: Unit of attribute 'x' must be equivalent to 'kpc'. Found 'g'.

## Automatic cohersion of units: Array Accessor

El mayor poder de *uttrs* es la capacidad de transformar de manera sencilla todas las unidades a `numpy.ndarray` planos, utilizando las unidades por defecto.

Para esto se provee de una funcion `uttr.array_accessor()` la cual permite acceder a los atributos definidos por *uttrs* de manera uniforme en una estructura de datos mas veloz que las que possen unidad.

Para agregar esta característica, se debe agregar un atributo extra a la clase que se iguale a `uttr.array_accessor()`. Se propone utilizar el nombre `arr_`

Extendiendo el ejemplo anterior

In [15]:
@attr.s
class Galaxy:
    x = uttr.ib(unit=u.kpc)
    y = uttr.ib(unit=u.kpc)
    z = uttr.ib(unit=u.kpc)

    vx = uttr.ib(unit=u.km / u.s)
    vy = uttr.ib(unit=u.km / u.s)
    vz = uttr.ib(unit=u.km / u.s)

    m = uttr.ib(unit=u.M_sun)

    notes = attr.ib(validator=attr.validators.instance_of(str))

    arr_ = uttr.array_accessor()  # el accessor

Ahora volvemos a instanciar la clase con algunos parámetros con unidades custom

In [16]:
gal = Galaxy(
    x=[1, 1, 3, 4],
    y=[10, 2, 3, 100],
    z=[1000, 1000, 1000, 1000] * u.parsec,
    vx=[1000, 1023, 2346, 1334],
    vy=[9956, 833, 954, 1024] * (u.km / u.h),
    vz=[1253, 956, 1054, 3568],
    m=[200, 100, 20, 5] * u.kg,
    notes="a random galaxy made with random numbers",
)

ahora si accedemos a `z` a travez de `arr_`, uttrs se encargara de convertir los parsecs en kiloparsecs y luego convertirlo en un numpy array

In [17]:
gal.z

<Quantity [1000., 1000., 1000., 1000.] pc>

Mientras que `z` mantiene sus unidades originales

In [18]:
gal.z

<Quantity [1000., 1000., 1000., 1000.] pc>

Lo mismo si accedemos a `vy` y `m`

In [19]:
gal.m

<Quantity [200., 100.,  20.,   5.] kg>

In [20]:
gal.vy

<Quantity [9956.,  833.,  954., 1024.] km / h>

Tratar de acceder a un atributo privado o que no sea un `uttr.ib`, lanza un error del tipo `AttributeError`

In [21]:
gal.notes

'a random galaxy made with random numbers'

## El uso de `array_accessor`

Es conocido que las unidades de Astropy son lentas cuando los calculos son complejos. 

Para evitar esto, los desarrolladores optan por unificar las unidades y luego convertir los valores a arrays de numpy para operarlos mas rapidamente; y al final se vuelve a asignas las unidades.

Para evitar esto, `array_accesor` realiza toda esta trasnformacion transparentemente para el usuario, evitando la necesidad de replicar informacion sobre las unidades.

Por ejemplo, si quisieramos programar un codigo que genere un nuevo objeto galaxia con una sola particula, promedio de las demas,
el código seria el siguiente:

In [22]:
@attr.s
class Galaxy:
    x = uttr.ib(unit=u.kpc)
    y = uttr.ib(unit=u.kpc)
    z = uttr.ib(unit=u.kpc)

    vx = uttr.ib(unit=u.km / u.s)
    vy = uttr.ib(unit=u.km / u.s)
    vz = uttr.ib(unit=u.km / u.s)

    m = uttr.ib(unit=u.M_sun)

    notes = attr.ib(validator=attr.validators.instance_of(str))

    arr_ = uttr.array_accessor()  # el accessor

    def mean(self):
        x = np.mean(self.x)
        y = np.mean(self.y)
        z = np.mean(self.z)

        vx = np.mean(self.vx)
        vy = np.mean(self.vy)
        vz = np.mean(self.vz)

        m = np.mean(self.m)

        return Galaxy(
            x=x, y=y, z=z, vx=vx, vy=vy, vz=vz, m=m, notes=self.notes
        )

Ahora podemos crear una galaxia de 1 millon de elementos aleatorios y calcular la galaxia "media"

In [23]:
import numpy as np

# fijamos la semilla random
random = np.random.default_rng(seed=42)

size = 1_000_000

gal = Galaxy(
    x=random.random(size=size),
    y=random.random(size=size),
    z=random.random(size=size) * u.parsec,
    vx=random.random(size=size),
    vy=random.random(size=size),
    vz=random.random(size=size) * (u.km / u.h),
    m=random.random(size=size) * u.kg,
    notes="a random galaxy made with random numbers",
)

In [24]:
gal.mean()

Galaxy(x=<Quantity 0.50002648 kpc>, y=<Quantity 0.49981983 kpc>, z=<Quantity 0.49982494 pc>, vx=<Quantity 0.49976787 km / s>, vy=<Quantity 0.50029902 km / s>, vz=<Quantity 0.50030066 km / h>, m=<Quantity 0.49949307 kg>, notes='a random galaxy made with random numbers')

Por completitud se ejemplifica a continuacion como seria el miemos codigo de `mean` pero sin usar `array_accessor`

In [25]:
@attr.s
class Galaxy:
    x = uttr.ib(unit=u.kpc)
    y = uttr.ib(unit=u.kpc)
    z = uttr.ib(unit=u.kpc)

    vx = uttr.ib(unit=u.km / u.s)
    vy = uttr.ib(unit=u.km / u.s)
    vz = uttr.ib(unit=u.km / u.s)

    m = uttr.ib(unit=u.M_sun)

    notes = attr.ib(validator=attr.validators.instance_of(str))

    arr_ = uttr.array_accessor()  # el accessor

    def mean(self):
        x = np.mean(self.x.to_value(u.kpc))
        y = np.mean(self.y.to_value(u.kpc))
        z = np.mean(self.z.to_value(u.kpc))

        vx = np.mean(self.vx.to_value(u.km / u.s))
        vy = np.mean(self.vy.to_value(u.km / u.s))
        vz = np.mean(self.vz.to_value(u.km / u.s))

        m = np.mean(self.m.to_value(u.M_sun))

        return Galaxy(
            x=x, y=y, z=z, vx=vx, vy=vy, vz=vz, m=m, notes=self.notes
        )

In [26]:
import datetime as dt
dt.datetime.now()

datetime.datetime(2020, 11, 21, 20, 23, 13, 899006)